In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy

## Language modeling

### Data

#### Description of Data

In [2]:
PATH='/home/wk/myProjects/data/Enron/tag/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

models/  test/  tmp/  train/


Let's look inside the training folder...

In [3]:
TRN

'/home/wk/myProjects/data/Enron/tag/train/'

In [4]:
trn_files = !ls {TRN}
#trn_files = !dir /w {TRN}
trn_files[7:17]

['allen_p_sent_120.txt',
 'allen_p_sent_125.txt',
 'allen_p_sent_134.txt',
 'allen_p_sent_137.txt',
 'allen_p_sent_142.txt',
 'allen_p_sent_156.txt',
 'allen_p_sent_159.txt',
 'allen_p_sent_15.txt',
 'allen_p_sent_161.txt',
 'allen_p_sent_164.txt']

In [5]:
trn_files[6]

'allen_p_sent_119.txt'

...and at an example review.

In [6]:
review = !cat {TRN}{trn_files[6]}
#review = !type {TRN}{trn_files[9]}
review[0]

'Message-ID: < @@othr_ph@@  @@othr_ph@@ .JavaMail.evans@thyme>'

Sounds like I'd really enjoy *Zombiegeddon*...

Now we'll check how many words are in the dataset.

In [7]:
!find {TRN} -name '*.txt' | xargs cat | wc -w

2116831


In [8]:
!find {VAL} -name '*.txt' | xargs cat | wc -w

324116


Before we can analyze text, we must first *tokenize* it. This refers to the process of splitting a sentence into an array of words (or more generally, into an array of *tokens*).

*Note:* If you get an error like:

    Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
    
then you need to install the Spacy language model by running this command on the command-line:

    $ python -m spacy download en

In [9]:
# conda install -c spacy spacy 
# python -m spacy download en
import spacy
spacy_tok = spacy.load('en')

In [10]:
' '.join([sent.string.strip() for sent in spacy_tok(review[0])])

'Message - ID : < @@othr_ph@@  @@othr_ph@@ .JavaMail.evans@thyme >'

We use Pytorch's [torchtext](https://github.com/pytorch/text) library to preprocess our data, telling it to use the wonderful [spacy](https://spacy.io/) library to handle tokenization.

First, we create a torchtext *field*, which describes how to preprocess a piece of text - in this case, we tell torchtext to make everything lowercase, and tokenize it with spacy.

In [11]:
TEXT = data.Field(lower=True)
#tokenize="spacy"

In [12]:
TEXT

fastai works closely with torchtext. We create a ModelData object for language modeling by taking advantage of `LanguageModelData`, passing it our torchtext field object, and the paths to our training, test, and validation sets. In this case, we don't have a separate test set, so we'll just use `VAL_PATH` for that too.

As well as the usual `bs` (batch size) parameter, we also now have `bptt`; this define how many words are processing at a time in each row of the mini-batch. More importantly, it defines how many 'layers' we will backprop through. Making this number higher will increase time and memory requirements, but will improve the model's ability to handle long sentences.

In [13]:
bs=32; bptt=75

In [14]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=200)

After building our `ModelData` object, it automatically fills the `TEXT` object with a very important attribute: `TEXT.vocab`. This is a *vocabulary*, which stores which words (or *tokens*) have been seen in the text, and how each word will be mapped to a unique integer id. We'll need to use this information again later, so we save it.

*(Technical note: python's standard `Pickle` library can't handle this correctly, so at the top of this notebook we used the `dill` library instead and imported it as `pickle`)*.

In [15]:
f'{PATH}models/TEXT.pkl'

'/home/wk/myProjects/data/Enron/tag/models/TEXT.pkl'

In [16]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

Here are the: # batches; # unique tokens in the vocab; # tokens in the training set; # sentences

In [17]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(886, 1014, 1, 2129380)

This is the start of the mapping from integer IDs to unique tokens.

In [18]:
# 'itos': 'int-to-string'
TEXT.vocab.itos[:12]

['<unk>',
 '<pad>',
 'the',
 '@@othr_em@@',
 'to',
 'and',
 '@@othr_tm@@',
 'of',
 '@@othr_ph@@',
 '@@firstname@@',
 '@@othr_dt@@',
 'a']

In [19]:
# 'stoi': 'string to int'
TEXT.vocab.stoi['the']

2

Note that in a `LanguageModelData` object there is only one item in each dataset: all the words of the text joined together.

In [20]:
md.trn_ds[0].text[:12]

['message-id:',
 '<',
 '@@othr_ph@@',
 '@@othr_ph@@',
 '85.javamail.evans@thyme>',
 'date:',
 'thu,',
 '@@othr_dt@@',
 '@@othr_tm@@',
 ':00',
 '-0800',
 '(pst)']

torchtext will handle turning this words into integer IDs for us automatically.

In [21]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

Variable containing:
   37
   20
    8
    8
    0
   31
  124
   10
    6
   48
   74
   75
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

Our `LanguageModelData` object will create batches with 64 columns (that's our batch size), and varying sequence lengths of around 80 tokens (that's our `bptt` parameter - *backprop through time*).

Each batch also contains the exact same data as labels, but one word later in the text - since we're trying to always predict the next word. The labels are flattened into a 1d array.

In [22]:
next(iter(md.trn_dl))

(Variable containing:
    37     9     0  ...     29    21     0
    20    21    40  ...      3    47   320
     8    47    37  ...     13   508     0
        ...          ⋱          ...       
     0    19    23  ...    124   374     3
    24   406     2  ...     10     0    12
    13   349     0  ...      6    26    29
 [torch.cuda.LongTensor of size 71x32 (GPU 0)], Variable containing:
    20
    21
    40
   ⋮  
    48
    23
    16
 [torch.cuda.LongTensor of size 2272 (GPU 0)])

### Train

We have a number of parameters to set - we'll learn more about these later, but you should find these values suitable for many problems.

In [23]:
em_sz = 300  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

Researchers have found that large amounts of *momentum* (which we'll learn about later) don't work well with these kinds of *RNN* models, so we create a version of the *Adam* optimizer with less momentum than it's default of `0.9`.

In [24]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

fastai uses a variant of the state of the art [AWD LSTM Language Model](https://arxiv.org/abs/1708.02182) developed by Stephen Merity. A key feature of this model is that it provides excellent regularization through [Dropout](https://en.wikipedia.org/wiki/Convolutional_neural_network#Dropout). There is no simple way known (yet!) to find the best values of the dropout parameters below - you just have to experiment...

However, the other parameters (`alpha`, `beta`, and `clip`) shouldn't generally need tuning.

In [25]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

As you can see below, I gradually tuned the language model in a few stages. I possibly could have trained it further (it wasn't yet overfitting), but I didn't have time to experiment more. Maybe you can see if you can train it to a better accuracy! (I used `lr_find` to find a good learning rate, but didn't save the output in this notebook. Feel free to try running it yourself now.)

In [26]:
learner.fit(5e-3, 2, wds=1e-6, cycle_len=2, cycle_mult=2)

epoch      trn_loss   val_loss                               
    0      2.582016   3.010363  
    1      2.398768   2.904188                               
    2      2.31833    2.872145                               
    3      2.252546   2.773089                               
    4      2.159381   2.70829                                
    5      2.125327   2.693057                               



[array([2.69306])]

In [27]:
learner.save_encoder('adam1_enc')

In [28]:
learner.load_encoder('adam1_enc')

In [29]:
learner.fit(4e-3, 1, wds=1e-6, cycle_len=8)

epoch      trn_loss   val_loss                               
    0      2.168133   2.756118  
    1      2.162371   2.728973                               
    2      2.099043   2.708253                               
    3      2.039913   2.69122                                
    4      2.036377   2.676881                               
    5      1.996784   2.668979                               
    6      1.956859   2.674985                               
    7      1.926043   2.663009                               



[array([2.66301])]

In [30]:
learner.save("rnn_enron")


In [31]:
learner.load("rnn_enron")

In the sentiment analysis section, we'll just need half of the language model - the *encoder*, so we save that part.

In [32]:
learner.save_encoder('adam3_10_enc')

In [33]:
learner.load_encoder('adam3_10_enc')

Language modeling accuracy is generally measured using the metric *perplexity*, which is simply `exp()` of the loss function we used.

In [34]:
math.exp(4.165)

64.3926824434624

In [35]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

### Test

We can play around with our language model a bit to check it seems to be working OK. First, let's create a short bit of text to 'prime' a set of predictions. We'll use our torchtext field to numericalize it so we can feed it to our language model.

In [36]:
m=learner.model

ss=       'Message-ID: <20914427.1075845987342.JavaMail.evans@thyme> ' 
ss = ss + 'Date: Wed, 21 Feb 2001 03:13:00 -0800 (PST) ' 
ss = ss + 'From: @@Email@@ '
ss = ss + 'To: nmann@erac.com ' 
ss = ss + 'Subject: RE: ' 
ss = ss + 'Mime-Version: 1.0 '
ss = ss + 'Content-Type: text/plain; charset=us-ascii '
ss = ss + 'Content-Transfer-Encoding: 7bit '
ss = ss + 'X-From: @@FirstName@@ @@LastName@@  '
ss = ss + 'X-To: muller@thedoghousemail.com '
ss = ss + 'X-cc: '
ss = ss + 'X-bcc: '
s = [TEXT.preprocess(ss)] 
t=TEXT.numericalize(s)
' '.join(s[0])

'message-id: <20914427.1075845987342.javamail.evans@thyme> date: wed, 21 feb 2001 03:13:00 -0800 (pst) from: @@email@@ to: nmann@erac.com subject: re: mime-version: 1.0 content-type: text/plain; charset=us-ascii content-transfer-encoding: 7bit x-from: @@firstname@@ @@lastname@@ x-to: muller@thedoghousemail.com x-cc: x-bcc:'

We haven't yet added methods to make it easy to test a language model, so we'll need to manually go through the steps.

In [37]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

Let's see what the top 10 predictions were for the next word after our short text:

In [38]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['x-folder:',
 'x-bcc:',
 '<unk>',
 '@@othr_tm@@',
 'message-----',
 '\\@@firstname@@_mann_june2001_3\\notes',
 '(pdt)',
 '-0800',
 'communications,',
 '1.0']

...and let's see if our model can generate a bit more text all by itself!

In [39]:
print(ss,"\n")
for i in range(200):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

Message-ID: <20914427.1075845987342.JavaMail.evans@thyme> Date: Wed, 21 Feb 2001 03:13:00 -0800 (PST) From: @@Email@@ To: nmann@erac.com Subject: RE: Mime-Version: 1.0 Content-Type: text/plain; charset=us-ascii Content-Transfer-Encoding: 7bit X-From: @@FirstName@@ @@LastName@@  X-To: muller@thedoghousemail.com X-cc: X-bcc:  

x-folder: \@@firstname@@_mann_june2001_3\notes folders\sent x-origin: mann-k x-filename: kmann.nsf i have a couple of questions about the other changes to the last two weeks and i have not heard back from you about this and i will be in the office on the next day or so to see the other time and the other day of the day and the first day of the day and the first day of the day and the first day of the day of the day of the day of the day of the day of the day of the conference call we will be in the office on the next day or so to discuss the issues and i will be in the office on thursday and will be able to attend the meeting on monday and will be able to attend t